In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import scipy
##import pandas_profiling
import tldextract

import matplotlib
import matplotlib.pyplot as plt # for plotting
import seaborn as sns # for making plots with seaborn

from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

import warnings
#import imblearn
warnings.filterwarnings("ignore")
import gc
import time
import json
from pandas.io.json import json_normalize

import os
print(os.listdir("../input"))
# Any results you write to the current directory are saved as output.

['sample_submission_v2.csv', 'test_v2.csv', 'train_v2.csv']


In [3]:
####################################################################################
#  Date :- 17/09/2018
#  Description :- Reusable Functions
#  Name :- Javed Sheikh
####################################################################################

def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))


def one_hot_encoder(data, nan_as_category = True):
    original_columns = list(data.columns)
    categorical_columns = [col for col in data.columns \
                           if not pd.api.types.is_numeric_dtype(data[col].dtype)]
    for c in categorical_columns:
        if nan_as_category:
            data[c].fillna('NaN', inplace = True)
        values = list(data[c].unique())
        for v in values:
            data[str(c) + '_' + str(v)] = (data[c] == v).astype(np.uint8)
    data.drop(categorical_columns, axis = 1, inplace = True)
    return data, [c for c in data.columns if c not in original_columns]

#pandas_profiling.ProfileReport(application_train)

def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])


def bar_plot_data(data,dataset):
    temp = dataset[data].value_counts()
    df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                  })
    plt.figure(figsize = (20,5))
    plt.title('Application loans repayed - train dataset')
    sns.barplot(x = 'labels', y="values", data=df)
    plt.show()

    
#Binning:
def binning(col, cut_points, labels=None):
  #Define min and max values:
  minval = col.min()
  maxval = col.max()


  #create list by adding min and max to cut_points
  break_points = [minval] + cut_points + [maxval]

  #if no labels provided, use default labels 0 ... (n-1)
  if not labels:
    labels = range(len(cut_points)+1)

  #Binning using cut function of pandas
  colBin = pd.cut(col,bins=break_points,labels=labels,include_lowest=True)
  return colBin
  

  # function to obtain Categorical Features
def _get_categorical_features(df):
    feats = [col for col in list(df.columns) if df[col].dtype == 'object']
    return feats

# function to factorize categorical features
def _factorize_categoricals(df, cats):
    for col in cats:
        df[col], _ = pd.factorize(df[col])
    return df 

# function to create dummy variables of categorical features
def _get_dummies(df, cats):
    for col in cats:
        df = pd.concat([df, pd.get_dummies(df[col], prefix=col)], axis=1)
    return df

import gc
gc.enable()
features = ['channelGrouping', 'date', 'fullVisitorId', 'visitId',\
       'visitNumber', 'visitStartTime', 'device.browser',\
       'device.deviceCategory', 'device.isMobile', 'device.operatingSystem',\
       'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country',\
       'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.region',\
       'geoNetwork.subContinent', 'totals.bounces', 'totals.hits',\
       'totals.newVisits', 'totals.pageviews', 'totals.transactionRevenue',\
       'trafficSource.adContent', 'trafficSource.campaign',\
       'trafficSource.isTrueDirect', 'trafficSource.keyword',\
       'trafficSource.medium', 'trafficSource.referralPath',\
       'trafficSource.source']


In [4]:
def load_df(csv_path='../input/train_v2.csv'):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    ans = pd.DataFrame()
    dfs = pd.read_csv(csv_path, sep=',',
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                    chunksize = 100000)
    for df in dfs:
        df.reset_index(drop = True,inplace = True)
        for column in JSON_COLUMNS:
            column_as_df = json_normalize(df[column])
            column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
            df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)

        print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
        use_df = df[features]
        del df
        gc.collect()
        ans = pd.concat([ans, use_df], axis = 0).reset_index(drop = True)
        print(ans.shape)
    return ans

train = load_df()
train.shape

Loaded train_v2.csv. Shape: (100000, 59)
(100000, 29)
Loaded train_v2.csv. Shape: (100000, 60)
(200000, 29)
Loaded train_v2.csv. Shape: (100000, 59)
(300000, 29)
Loaded train_v2.csv. Shape: (100000, 59)
(400000, 29)
Loaded train_v2.csv. Shape: (100000, 59)
(500000, 29)
Loaded train_v2.csv. Shape: (100000, 59)
(600000, 29)
Loaded train_v2.csv. Shape: (100000, 59)
(700000, 29)
Loaded train_v2.csv. Shape: (100000, 59)
(800000, 29)
Loaded train_v2.csv. Shape: (100000, 59)
(900000, 29)
Loaded train_v2.csv. Shape: (100000, 59)
(1000000, 29)
Loaded train_v2.csv. Shape: (100000, 59)
(1100000, 29)
Loaded train_v2.csv. Shape: (100000, 59)
(1200000, 29)
Loaded train_v2.csv. Shape: (100000, 59)
(1300000, 29)
Loaded train_v2.csv. Shape: (100000, 59)
(1400000, 29)
Loaded train_v2.csv. Shape: (100000, 59)
(1500000, 29)
Loaded train_v2.csv. Shape: (100000, 59)
(1600000, 29)
Loaded train_v2.csv. Shape: (100000, 59)
(1700000, 29)
Loaded train_v2.csv. Shape: (8337, 59)
(1708337, 29)


(1708337, 29)

In [5]:
####################################################################################
#  Date :- 17/09/2018
#  Description :- Reading Data
#  Name :- Javed Sheikh
####################################################################################

##train_df = load_df("../input/train_v2.csv")
test_df = load_df("../input/test_v2.csv")

Loaded test_v2.csv. Shape: (100000, 59)
(100000, 29)
Loaded test_v2.csv. Shape: (100000, 59)
(200000, 29)
Loaded test_v2.csv. Shape: (100000, 59)
(300000, 29)
Loaded test_v2.csv. Shape: (100000, 59)
(400000, 29)
Loaded test_v2.csv. Shape: (1589, 59)
(401589, 29)


In [6]:
train_df = train
del train
gc.collect()

0

In [10]:
test_df.columns.isin(train_df.columns)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [ ]:

train_df['totals.transactionRevenue'] = train_df['totals.transactionRevenue'].fillna(0).astype(float)

plt.figure(figsize=(15,15))
for i, binwidth in enumerate([1, 2, 3, 4]):
    # Set up the plot
    ax = plt.subplot(2, 2, i + 1)
    # Draw the plot
    ax.hist(np.log1p(train_df.loc[train_df['totals.transactionRevenue'] > 0,'totals.transactionRevenue']), 
            bins = int(180/binwidth)
            ,color = 'Green'
            ,edgecolor = 'white'
           )
    # Title and labels
    ax.set_title('Histogram with Binwidth = %d' % binwidth, size = 30)
    ax.set_xlabel('Target Revenue', size = 22)
    ax.set_ylabel('Count', size= 22)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
sns.distplot((train_df.loc[train_df['totals.transactionRevenue'] > 0,'totals.transactionRevenue'])
             ,hist=True
             ,kde=True
             ,bins=100
             ,color = 'darkblue'
             ,hist_kws={'edgecolor':'white'}
             ,kde_kws={'linewidth': 4}
            )

plt.figure(figsize=(12,6))
sns.distplot(np.log1p(train_df.loc[train_df['totals.transactionRevenue'] > 0,'totals.transactionRevenue'])
             ,hist=True
             ,kde=True
             ,bins=100
             ,color = 'Green'
             ,hist_kws={'edgecolor':'white'}
             ,kde_kws={'linewidth': 4}
            )

In [ ]:
train_df.head()

**Merging Dataset**

In [ ]:
train_df.columns.difference(test_df.columns)
#train_df.drop('trafficSource.campaignCode', axis = 1,inplace=True)
train_df['totals.transactionRevenue'] = np.log1p(train_df['totals.transactionRevenue'].fillna(0).astype(float))

#Flaging Test / train set
test_df['is_train'] = 0
train_df['is_train'] = 1
test_df['is_test'] = 1
train_df['is_test'] = 0

###Target Variables
Y = train_df['totals.transactionRevenue']
train = train_df.drop(['totals.transactionRevenue'], axis = 1)
test =  test_df

# Merge train and test
data = pd.concat([train, test], axis=0)

In [ ]:
data['trafficSource.adContent'] = data['trafficSource.adContent'].replace({'/':''}, regex=True).astype(str)
data['trafficSource.adContent'] = data['trafficSource.adContent'].replace({'}':''}, regex=True)
data['trafficSource.adContent'] = data['trafficSource.adContent'].replace({'{KeyWord:':''}, regex=True)
## Replacing Missing values
data['trafficSource.adContent'] = data['trafficSource.adContent'].fillna('Unknown')
data['trafficSource.isTrueDirect'] = data['trafficSource.isTrueDirect'].fillna('False')
data['trafficSource.referralPath'] = data['trafficSource.referralPath'].fillna('/')
data['trafficSource.keyword'] = data['trafficSource.keyword'].fillna('(not provided)')
data['totals.bounces'] = data['totals.bounces'].fillna(0).astype(int)
data['totals.newVisits'] = data['totals.newVisits'].fillna(0).astype(int)
data['totals.pageviews'] = data['totals.pageviews'].fillna(0).astype(int)
data['totals.transactionRevenue'] = data['totals.transactionRevenue'].fillna(0)

In [ ]:
missing_data(data)

In [ ]:
import category_encoders

In [ ]:
#########################################
## Cleaning the URL's
#########################################
no_fetch_extract = tldextract.TLDExtract(suffix_list_urls=None)
## Removing the numbers and ":" if any
data['trafficSource.source_mod'] = data['trafficSource.source'].str.replace('\d+', '').str.replace(':', '')
## Use tldextract lib to clean the Source URL
data['Domain'] = data['trafficSource.source_mod'].apply(lambda x: no_fetch_extract(x).domain)
data['suffix'] = data['trafficSource.source_mod'].apply(lambda x: no_fetch_extract(x).suffix)
data['trafficSource.source'] = np.where(data['Domain'] == '', data['suffix'], data['Domain'])
## Drop temporory columns
data.drop(['trafficSource.source_mod','Domain','suffix'], axis=1,inplace = True)

In [ ]:
MAP_DICT={
'Chrome':'Chrome',
'Firefox':'Firefox',
'UC Browser':'UCBrowser',
'Internet Explorer':'InternetExplorer',
'Safari':'Safari',
'Edge':'InternetExplorer',
'Opera Mini':'Opera',
'Opera':'Opera',
'BlackBerry':'BlackBerry',
'Safari (in-app)':'Safari',
'Coc Coc':'CocCoc',
'Mozilla Compatible Agent':'Firefox',
'ADM':'Android',
'MRCHROME':'Chrome',
'Amazon Silk':'Amazon',
'YaBrowser':'YaBrowser',
'Android Webview':'Android',
'Puffin':'Puffin',
'Nokia Browser':'Nokia',
'Maxthon':'Maxthon',
'Nintendo Browser':'Nintendo',
'Android Browser':'Android',
'Lunascape':'Others',
'IE with Chrome Frame':'InternetExplorer',
'ThumbSniper':'Others',
'LYF_LS_4002_12':'Others',
'Mozilla':'Firefox',
'osee2unifiedRelease':'Others',
'NokiaE52-1':'Nokia',
'Iron':'Iron',
'[Use default User-agent string] LIVRENPOCHE':'Others',
'(not set)':'Others',
'LYF_LS_4002_11':'Others',
'M5':'M5',
'Android Runtime':'Android',
'Apple-iPhone7C2':'Safari',
'SeaMonkey':'SeaMonkey',
'Konqueror':'Others',
'Seznam':'Seznam',
'Changa 99695759':'Others',
'no-ua':'Others',
'MQQBrowser':'MQQ',
'Nichrome':'Nichrome',
'HTC802t_TD':'HTC',
'DASH_JR_3G':'DASH',
'DoCoMo':'DoCoMo',
'subjectAgent: NoticiasBoom':'Others',
'YE':'YE',
'User Agent':'Others',
'0':'Others',
'Hisense M20-M_LTE':'Hisense',
'Reddit':'Reddit',
'TCL P500M':'TCL',
'CSM Click':'CSM'}   
data['device.browser'] = data['device.browser'].map(MAP_DICT)

In [ ]:
data['totals.bounces'] = data['totals.bounces'].astype(int)
data['totals.pageviews'] = data['totals.pageviews'].astype(int)
data['totals.newVisits'] = data['totals.newVisits'].astype(int)
data['totals.hits'] = data['totals.hits'].astype(int)
data['totals.Activity'] = np.log1p(data['totals.hits']  + data['totals.pageviews'] + data['totals.newVisits'] + data['totals.bounces'] )

In [ ]:
bins = [0, 1, 2, 3, 4, 5, 6, 7,8]
labels = [1,2,3,4,5,6,7,8]
data['Level of Activity'] = pd.cut(data['totals.Activity'], bins=bins, labels=labels).astype(int)

In [ ]:
'''

#data.drop('totals.visits', axis = 1,inplace=True)
#data.drop('totals.pageviews', axis = 1,inplace=True)
#data.drop('totals.newVisits', axis = 1,inplace=True)
#data.drop('totals.hits', axis = 1,inplace=True)
#data.drop('totals.bounces', axis = 1,inplace=True)
data.drop('totals.Activity', axis = 1,inplace=True)
data.drop(['trafficSource.adwordsClickInfo.gclId','geoNetwork.cityId','device.screenResolution',
           'device.screenColors','device.operatingSystemVersion','device.mobileInputSelector',
           'device.mobileDeviceModel','device.mobileDeviceMarketingName','device.mobileDeviceInfo',
           'device.mobileDeviceBranding','device.language','device.flashVersion','device.browserVersion',
           'device.browserSize','geoNetwork.latitude','geoNetwork.longitude']
          , axis=1, inplace = True)

'''

In [ ]:
data['date'] = data['date'].astype(str).apply(lambda x : x[:4] + "-" + x[4:6] + "-" + x[6:])
data['date'] = pd.to_datetime(data["date"])

In [ ]:
data['date_MM']  = data['date'].dt.month
data['date_D']   = data['date'].dt.day
data['date_Day'] = data['date'].dt.weekday
data['hour'] = data['date'].dt.hour


data["_id_incoherence"] = pd.to_datetime(data['visitId'], unit='s') != data['date']

from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
cal = calendar()
holidays = cal.holidays(start=data['date'].min(), end=data['date'].max())
data['is_holiday'] = data['date'].dt.date.astype('datetime64').isin(holidays)

In [ ]:
  
# remember visitId dublicates?
data["_visitId_dublicates"] = data.visitId.map(data.visitId.value_counts())


In [ ]:
data['source.country'] = data['trafficSource.source'] + '_' + data['geoNetwork.country']
data['campaign.medium'] = data['trafficSource.campaign'] + '_' + data['trafficSource.medium']
data['browser.category'] = data['device.browser'] + '_' + data['device.deviceCategory']
data['browser.os'] = data['device.browser'] + '_' + data['device.operatingSystem']
data['device_deviceCategory_channelGrouping'] = data['device.deviceCategory'] + "_" + data['channelGrouping']
data['channelGrouping_browser'] = data['device.browser'] + "_" + data['channelGrouping']
data['channelGrouping_OS'] = data['device.operatingSystem'] + "_" + data['channelGrouping']
data['content.source'] = data['trafficSource.adContent'].astype(str) + "_" + data['source.country']
data['medium.source'] = data['trafficSource.medium'] + "_" + data['source.country']
data['visitStartTime_Time'] = pd.to_datetime(data['visitStartTime'], unit='s')
data['visitStartTime_HHMMSS'] = data['visitStartTime_Time'].astype(str).apply(lambda x:x.split(' ')[1])
data['visitStartTime_HH'] = data['visitStartTime_HHMMSS'].astype(str).apply(lambda x:x.split(':')[0])
data['visitStartTime_MM'] = data['visitStartTime_HHMMSS'].astype(str).apply(lambda x:x.split(':')[1])
data.drop(['visitStartTime_Time','visitId'], axis = 1,inplace=True)


In [ ]:
plt.figure(figsize=(20,6))
data.loc[data['is_test'] == 1,'date'].value_counts().sort_index().plot(label="test", color = "#FFD700")
data.loc[data['is_train'] == 1,'date'].value_counts().sort_index().plot(label="train", color = "#48D1CC")

In [ ]:
bins = [0,10,20,30,40,50,80,120,200,300,800]
labels = [1,2,3,4,5,6,7,8,9,10]
data['visitNumber'] = pd.cut(data['visitNumber'], bins=bins, labels=labels).astype(int)

In [ ]:
data['nb_pageviews'] = data['date'].map( data[['date', 'totals.pageviews']].groupby('date')['totals.pageviews'].sum()    )
data['ratio_pageviews'] = data['totals.pageviews'] / data['nb_pageviews']

data['date'] = data['date'].astype(int)
data['visitStartTime_HH'] = data['visitStartTime_HH'].astype(int)
data['visitStartTime_MM'] = data['visitStartTime_MM'].astype(int)

In [ ]:
# get categorical features
data_cats = _get_categorical_features(data)
data_cats.remove('fullVisitorId')

# factorize the categorical features from train and test data
data = _factorize_categoricals(data, data_cats)
data.drop(['date'], axis = 1,inplace=True)
data.reset_index(drop=True,inplace=True)

In [ ]:
data_cats

In [ ]:
ignore_features = ['is_train', 'is_test']
relevant_features = [col for col in data.columns if col not in ignore_features]
trainX = data[data['is_train'] == 1][relevant_features]
testX = data[data['is_test'] == 1][relevant_features]

test_id = testX['fullVisitorId']
train_id = trainX['fullVisitorId']

trainX.drop(['fullVisitorId'], axis = 1,inplace=True)
testX.drop(['fullVisitorId'], axis = 1,inplace=True)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(trainX, Y, test_size=0.3, random_state=18)
lgb_train = lgb.Dataset(data=x_train, label=y_train)
lgb_eval = lgb.Dataset(data=x_val, label=y_val)

In [ ]:

params = {
        'learning_rate': 0.02,
       # 'boosting_type':'gbdt',
        'objective':'regression',
        'metric':'rmse',
        'num_leaves': 31,
        'verbose': 1,
        'bagging_fraction': 0.7934712636944741,
        'feature_fraction': 0.686612409641711,
        "random_state":42,
        'max_depth': 5,
        "bagging_seed" : 2019,
        "verbosity" : -1,
        "bagging_frequency" : 6,
        'lambda_l2': 0.02085548700474218,
        'lambda_l1': 0.004107624022751344,
        'min_child_samples': 21,
        'use_best_model':True,
        'min_child_samples': 21
         }
model = lgb.train(params, 
                  lgb_train,
                  categorical_feature=['channelGrouping', 'device.browser', 'device.deviceCategory', 'device.operatingSystem', 'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country',
 'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.region', 'geoNetwork.subContinent', 'totals.transactionRevenue', 'trafficSource.adContent', 'trafficSource.campaign', 'trafficSource.isTrueDirect',
 'trafficSource.keyword', 'trafficSource.medium', 'trafficSource.referralPath', 'trafficSource.source', 'source.country', 'campaign.medium', 'browser.category', 'browser.os', 'device_deviceCategory_channelGrouping',
 'channelGrouping_browser', 'channelGrouping_OS', 'content.source', 'medium.source', 'visitStartTime_HHMMSS'],
                  valid_sets=lgb_eval, 
                  num_boost_round=10000,
                  early_stopping_rounds=200 ,
                  verbose_eval=400
                 )

In [ ]:
Data_T = lgb.Dataset(data=trainX, label=Y)

In [ ]:
params = {
        'learning_rate': 0.03,
        'boosting_type':'gbdt',
        'objective':'regression',
        'metric':'rmse',
        'num_leaves': 31,
        'verbose': 1,
        'bagging_fraction': 0.75,
        'feature_fraction': 0.55,
        "random_state":42,
        'max_depth': 5,
        "bagging_seed" : 2019,
        "verbosity" : -1,
        "bagging_frequency" : 6,
        'lambda_l2': 0.5,
        'lambda_l1': 0.5,
        'min_child_samples': 100,
        'use_best_model':True
         }
model_lgb = lgb.train(params, 
                  Data_T,
                  categorical_feature=['channelGrouping', 'device.browser', 'device.deviceCategory', 'device.operatingSystem', 'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country',
 'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.region', 'geoNetwork.subContinent', 'totals.transactionRevenue', 'trafficSource.adContent', 'trafficSource.campaign', 'trafficSource.isTrueDirect',
 'trafficSource.keyword', 'trafficSource.medium', 'trafficSource.referralPath', 'trafficSource.source', 'source.country', 'campaign.medium', 'browser.category', 'browser.os', 'device_deviceCategory_channelGrouping',
 'channelGrouping_browser', 'channelGrouping_OS', 'content.source', 'medium.source', 'visitStartTime_HHMMSS'],
                 # valid_sets=lgb_eval, 
                  num_boost_round=2000,
                 # early_stopping_rounds=200, 
                  verbose_eval=400)

In [ ]:
lgb.plot_importance(model_lgb, figsize=(12, 25), max_num_features=600)

In [ ]:
lgb_preds = model.predict(testX)
lgb_preds

In [ ]:
#test_id = testX['fullVisitorId']  np.expm1(Y)
sub_lgb = pd.DataFrame()
sub_lgb['fullVisitorId'] = test_id
sub_lgb['PredictedLogRevenue'] = lgb_preds
#sub_lgb['PredictedLogRevenue'] = np.expm1(sub_lgb['PredictedLogRevenue'])
sub_lgb = sub_lgb.groupby("fullVisitorId")["PredictedLogRevenue"].sum().reset_index()

#sub_lgb.to_csv("lgb_baseline.csv", index=False)


sub_lgb.head()

In [ ]:
##https://www.kaggle.com/rahullalu/gstore-eda-lgbm-baseline-1-4260
#READING SUMISSION FILE
#DATASET VIEW
path1="../input/"

submission=pd.read_csv(path1+'sample_submission_v2.csv')

#CREATING JOIN BETWEEN PREDICTED DATA WITH SUBMISSION FILE
submission=submission.join(sub_lgb.set_index('fullVisitorId'),on='fullVisitorId',lsuffix='_sub')
submission.drop('PredictedLogRevenue_sub',axis=1,inplace=True)

#HANDLING NaN IN CASE OF MISSING fullVisitorId
submission.fillna(0,inplace=True)

#SUBMITING FILE
submission.to_csv('LGBM_submission.csv',index=False)